General Imports

In [1]:
import numpy as np 
import pandas as pd 
import gensim
from gensim.test.utils import datapath
from gensim import utils
from gensim import models
import contractions
import string
import re
import nltk
import re
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support


Extraction of Dataset

In [2]:
df1 = pd.read_csv('/kaggle/input/data11/dataset/JDT_new.csv')
df2 = pd.read_csv('/kaggle/input/data11/dataset/PDE_new.csv')
df3 = pd.read_csv('/kaggle/input/data11/dataset/bugzilla_new.csv')
df4 = pd.read_csv('/kaggle/input/data11/dataset/cdt_1_new.csv')
df5 = pd.read_csv('/kaggle/input/data11/dataset/firefox_new.csv')
df6 = pd.read_csv('/kaggle/input/data11/dataset/thunderbird_new.csv')

In [3]:
df_list=[df1, df2, df3 ,df4, df5, df6]

In [4]:
for df in df_list:
    df.drop(columns='Unnamed: 0', axis=1, inplace=True, errors='raise')

In [5]:
new_cols = ['desc','severity']  

for df in df_list:
    df.columns = new_cols

In [6]:
lst=[]
for i in range(len(df1)):
    lst.append('jdt')
prd=pd.DataFrame(lst,columns=['product'])
df1=pd.concat([prd,df1],axis=1)

lst=[]
for i in range(len(df2)):
    lst.append('pde')
prd=pd.DataFrame(lst,columns=['product'])
df2=pd.concat([prd,df2],axis=1)


lst=[]
for i in range(len(df3)):
    lst.append('bugzilla')
prd=pd.DataFrame(lst,columns=['product'])
df3=pd.concat([prd,df3],axis=1)


lst=[]
for i in range(len(df4)):
    lst.append('cdt')
prd=pd.DataFrame(lst,columns=['product'])
df4=pd.concat([prd,df4],axis=1)

lst=[]
for i in range(len(df5)):
    lst.append('firefox')
prd=pd.DataFrame(lst,columns=['product'])
df5=pd.concat([prd,df5],axis=1)

lst=[]
for i in range(len(df6)):
    lst.append('thunderbird')
prd=pd.DataFrame(lst,columns=['product'])
df6=pd.concat([prd,df6],axis=1)

In [7]:
cdf = pd.concat([df1, df2, df3, df4, df5, df6], axis=0,ignore_index=True)
cdf['desc']=cdf['desc'].astype(str)

In [8]:
cdf['product'].value_counts()

product
jdt            3820
thunderbird    3293
bugzilla       2848
firefox        2824
cdt            2550
pde            1873
Name: count, dtype: int64

Preprocessing

In [9]:
cdf=cdf.dropna(axis=0,how='any')

In [ ]:
!pip install contractions

Needless text removal

In [12]:
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r' ',text)

def strip_newlines(text):
    return text.replace("\n", " ").replace("\r", " ").replace("\t"," ")

def expand_text(text):
    return contractions.fix(text)

punctuations=string.punctuation
punct=[punc for punc in punctuations]
def remove_punct(text):
    for punc in punct:
        text=text.replace(punc,' ')
    return text

In [13]:
cdf['desc']=cdf['desc'].apply(lambda x:remove_url(str(x)))
cdf['desc']=cdf['desc'].apply(lambda x:strip_newlines(str(x)))
cdf['desc']=cdf['desc'].apply(lambda x:expand_text(str(x)))
cdf['desc']=cdf['desc'].apply(lambda x:remove_punct(str(x)))
cdf['desc']=cdf['desc'].str.lower()

In [ ]:
nltk.download('punkt')
nltk.download('corpus')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download("words")

In [15]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [16]:
stopwords=set(stopwords.words('english'))
stopwords.remove('not')
# remove stopwords
def remove_sw(text):
    new_text=[]
    for word in text.split():
        if word not in stopwords:
            new_text.append(word)
    return " ".join(new_text)

In [17]:
def remove_alphanum(words):
    new_words=[]
    pat=re.compile("^[A-Za-z]+$")
    for word in words.split():
        if pat.fullmatch(word) is not None:
            new_words.append(word)
    return " ".join(new_words)

In [18]:
cdf['desc']=cdf['desc'].apply(lambda x:remove_sw(str(x)))
cdf['desc']=cdf['desc'].apply(lambda x:remove_alphanum(str(x)))

In [ ]:
%cd /usr/share/nltk_data/corpora/

In [ ]:
!unzip wordnet.zip

Lemmatization

In [22]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
# import pattern
# from pattern.en import lemma, lexeme

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_A(sentence):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
    return " ".join([lemma(wd) for wd in sentence.split()])


In [23]:
cdf['desc']=cdf['desc'].apply(lambda x:lemmatize_A(str(x)))

In [ ]:
max([len(sentence.split())  for sentence in cdf['desc']])

RoBERTa Model

In [ ]:
!pip install transformers

In [ ]:
!pip install scipy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

In [73]:
tweet_proc = 'java stopped working'
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

In [74]:
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [75]:
labels = ['negative', 'neutral', 'positive']

In [76]:
for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

negative 0.73579836
neutral 0.23445661
positive 0.02974502


Word2Vec model:-

In [24]:
# class MyCorpus():
#     """An iterator that yields sentences (lists of str)."""
#     def __iter__(self):
#         corpus_path = datapath("/kaggle/input/data12/combined_corpus.txt")
#         for line in open(corpus_path):
#             # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)

In [ ]:
import multiprocessing
cores=multiprocessing.cpu_count()
print(cores)

In [26]:
# sentences2 = MyCorpus()
# model2 = gensim.models.Word2Vec(sentences=sentences2,vector_size=300, min_count=3,window=4,workers=cores-1)

In [ ]:
# from joblib import Parallel, delayed 
# import joblib 


# # Save the model as a pickle in a file 
# joblib.dump(model2, '/kaggle/working/model2.pkl') 

# Load the model from the file 
# knn_from_joblib = joblib.load('filename.pkl') 

# # Use the loaded model to make predictions 
# knn_from_joblib.predict(X_test) 


# Load the model from the file 
# model2 = joblib.load('savedmodel.pkl') 

# Use the loaded model to make predictions 
# knn_from_joblib.predict(X_test) 


In [78]:
from joblib import Parallel, delayed 
import joblib 

model2 = joblib.load('/kaggle/input/model2/model2.pkl') 

In [ ]:
model2.wv.most_similar(positive=["good"])

Binary Classification:-    
['minor', 'trivial', 'normal'] => 'non-severe'       
['major','critical','blocker'] => 'severe'

In [80]:
cdf['severity'] = ['non-severe' if x in ['minor', 'trivial', 'normal'] else 'severe' for x in cdf['severity']]

Emotional Score Calculation 

In [132]:
def es(text, model):
    encoded_tweet = tokenizer(text, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    return scores[2]-scores[0]

In [133]:
cdf['es'] = [es(text, model) for text in cdf['desc']]

In [136]:
cdf.to_csv('/kaggle/working/data.csv')

Separating different Products

In [160]:
datadict={}
for prd in cdf['product'].unique():
    filtered_df=cdf[cdf['product']==prd]
    datadict[prd]=[filtered_df.iloc[:,1].to_list(),filtered_df.iloc[:,2].to_list(), filtered_df.iloc[:,3].to_list()]

In [214]:
cdf['severity'].value_counts()

severity
severe        11392
non-severe     5816
Name: count, dtype: int64

In [147]:
"""Function for Vectorization of Bug report description using Word2Vec"""
unknown=np.zeros(300)
def pad_X(X,W2V,max_length=36):
    uk_cnt=0
    X_Vec =[]
    for word in X:
        if word in W2V:
            X_Vec.append(W2V[word])
        else:
            uk_cnt+=1
            X_Vec.append(unknown)
    X_Vec = np.array(X_Vec)
    length = max_length - X_Vec.shape[0]
    pad = np.zeros(shape=(length, 300))
    X_Vec = np.concatenate([X_Vec, pad])
    return X_Vec, uk_cnt

In [148]:
def sev2int(text):
    if(str(text)=='severe'):
        return 1
    return 0
def es2int(text):
    if(str(text)=='positive'):
        return 1
    elif(str(text)=='negative'):
        return -1
    else:
        return 0

In [211]:
"""Function generating Test Data"""
def generate_test(prd,chunk_size,data,model):
    X_test_chunk = []
    X_test_es_chunk=[]
    y_test_chunk = []
    val=data[prd]
    for i in range(len(val[0])):
        if(len(str(val[0][i]))>0):
            X_test_chunk.append(str(val[0][i]))
            X_test_es_chunk.append(np.array(val[2][i]).reshape(1, 1))
            y_test_chunk.append(val[1][i])
        if len(X_test_chunk) == chunk_size or i==len(val[0])-1:
            X_test = np.array([pad_X(word_tokenize(text), model.wv)[0] for text in X_test_chunk])
            X_test_es=np.array(X_test_es_chunk)
            y_test = np.array([sev2int(val) for val in y_test_chunk])
            return X_test_es, X_test, y_test

In [192]:
"""Function Generating Train Data"""
def generate_train(prd,chunk_size,data, model2):
    X_train_chunk = []
    y_train_chunk = []
    X_train_es_chunk =[]
    combined=[[],[], []]
    for key,val in data.items():
        if(key!=prd):
            combined[0]+=val[0]
            combined[1]+=val[1]
            combined[2]+=[np.array(x).reshape(1, 1) for x in val[2]]
            
    for i in range(len(combined[0])):
        if(len(str(combined[0][i]))>0):
            X_train_chunk.append(str(combined[0][i]))
            y_train_chunk.append(combined[1][i])
            X_train_es_chunk.append(combined[2][i])
        if len(X_train_chunk) == chunk_size or i==len(combined[0])-1:
            X_train = np.array([pad_X(word_tokenize(text), model2.wv)[0] for text in X_train_chunk])
            y_train = np.array([sev2int(val) for val in y_train_chunk])
            X_train_es = np.array(X_train_es_chunk)
            # yield X_train_chunk, y_train_chunk  
            return X_train_es, X_train,y_train
            X_train_chunk = []  
            y_train_chunk = []  

Neural Network Models

In [222]:
"""LSTM Model"""
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Conv1D,Conv2D,Flatten, Dense, concatenate, MaxPooling1D, Dropout,Permute
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_validate

def create_model():
    
    input1_shape = (1,1)
    input2_shape = (36, 300)

    input1 = Input(shape=input1_shape, name='input1')

    layer_es1 = Conv1D(filters=128, kernel_size=1, activation='tanh')(input1)
    layer_es2 = Conv1D(filters=128, kernel_size=1, activation='tanh', kernel_regularizer=regularizers.l2(0.01))(layer_es1)
    layer_es3 = Conv1D(filters=128, kernel_size=1, activation='tanh')(layer_es2)
    flatten_output1 = Flatten()(layer_es3)

    input2 = Input(shape=input2_shape, name='input2')

    layer_1 = layers.LSTM(64, return_sequences=True)(input2)
    d1 = layers.Dropout(0.2)(layer_1)
    layer_2 = layers.LSTM(64, return_sequences=True)(d1)
    d2 = layers.Dropout(0.2)(layer_2)
    layer_3 = layers.LSTM(64, return_sequences=True)(d2)
    d3 = layers.Dropout(0.2)(layer_3)

    flatten_output2 = Flatten()(d3)
    
    concatenated_outputs = concatenate([flatten_output1, flatten_output2])

    dense_layer1 = Dense(units=128, activation='tanh', kernel_regularizer=regularizers.l2(0.01))(concatenated_outputs)
    output_layer = Dense(units=1, activation='sigmoid')(dense_layer1)

    model = tf.keras.Model(inputs=[input1, input2], outputs=output_layer)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model
    METRICS = [
          keras.metrics.TruePositives(name='tp'),
          keras.metrics.FalsePositives(name='fp'),
          keras.metrics.TrueNegatives(name='tn'),
          keras.metrics.FalseNegatives(name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
    return model
    

In [193]:
"""CNN Model"""
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv1D,Conv2D,Flatten, Dense, concatenate, MaxPooling1D, Dropout,Permute
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_validate

# def create_model1():
def create_model1():
    input1_shape = (1,1)
    input2_shape = (36, 300)

    input1 = Input(shape=input1_shape, name='input1')

    layer_es1 = Conv1D(filters=128, kernel_size=1, activation='tanh')(input1)
    layer_es2 = Conv1D(filters=128, kernel_size=1, activation='tanh', kernel_regularizer=regularizers.l2(0.01))(layer_es1)
    layer_es3 = Conv1D(filters=128, kernel_size=1, activation='tanh')(layer_es2)
    flatten_output1 = Flatten()(layer_es3)

    input2 = Input(shape=input2_shape, name='input2')

    layer_1 = Conv1D(filters=128, kernel_size=3, activation='tanh')(input2)
    layer_2 = Conv1D(filters=128, kernel_size=3, activation='tanh', kernel_regularizer=regularizers.l2(0.01))(layer_1)
    layer_3 = Conv1D(filters=128, kernel_size=3, activation='tanh')(layer_2)
    flatten_output2 = Flatten()(layer_3)

    concatenated_outputs = concatenate([flatten_output1, flatten_output2])

    dense_layer1 = Dense(units=128, activation='tanh', kernel_regularizer=regularizers.l2(0.01))(concatenated_outputs)
    output_layer = Dense(units=1, activation='sigmoid')(dense_layer1)

    model = tf.keras.Model(inputs=[input1, input2], outputs=output_layer)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model
    METRICS = [
          keras.metrics.TruePositives(name='tp'),
          keras.metrics.FalsePositives(name='fp'),
          keras.metrics.TrueNegatives(name='tn'),
          keras.metrics.FalseNegatives(name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.Precision(name='precision'),
          keras.metrics.Recall(name='recall'),
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
    return model

In [223]:
model = create_model()
model.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input2 (InputLayer) │ (None, 36, 300)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 36, 64)    │     93,440 │ input2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 36, 64)    │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input1 (InputLayer) │ (None, 1, 1)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 36, 64)    │     33,024 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_108 (Conv1D) │ (None, 1, 128)    │        256 │ input1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 36, 64)    │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_109 (Conv1D) │ (None, 1, 128)    │     16,512 │ conv1d_108[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 36, 64)    │     33,024 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_110 (Conv1D) │ (None, 1, 128)    │     16,512 │ conv1d_109[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 36, 64)    │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_36          │ (None, 128)       │          0 │ conv1d_110[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_37          │ (None, 2304)      │          0 │ dropout_2[0][0]   │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 2432)      │          0 │ flatten_36[0][0], │
│ (Concatenate)       │                   │            │ flatten_37[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 128)       │    311,424 │ concatenate_18[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 1)         │        129 │ dense_36[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 504,321 (1.92 MB)

 Trainable params: 504,321 (1.92 MB)

 Non-trainable params: 0 (0.00 B)

Training and Evaluation

In [225]:
for prd in cdf['product'].unique():

    prd ='jdt'

    model = create_model()

    X_train1, X_train2 ,y_train = generate_train(prd,50000,datadict, model2)

    neg, pos = np.bincount(y_train)
    total = neg + pos

    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    model.fit([X_train1, X_train2], y_train, epochs=15, batch_size=64, class_weight = class_weight)

    X_test1, X_test2,y_test=generate_test(prd,20000,datadict,model2)

    test_predictions = (model.predict([X_test1, X_test2]) > 0.5).astype(int)

    print(classification_report(y_test, test_predictions))
    #     print(precision_recall_fscore_support(y_test, test_predictions))
    

Epoch 1/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.7244 - auc: 0.8122 - fn: 1313.0901 - fp: 460.7962 - loss: 1.6935 - prc: 0.9077 - precision: 0.8655 - recall: 0.7154 - tn: 1632.2512 - tp: 3377.0569
Epoch 2/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7793 - auc: 0.8781 - fn: 1169.7820 - fp: 336.5829 - loss: 0.4426 - prc: 0.9425 - precision: 0.9132 - recall: 0.7502 - tn: 1804.3981 - tp: 3472.4314
Epoch 3/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7960 - auc: 0.8915 - fn: 1062.4739 - fp: 314.2133 - loss: 0.4179 - prc: 0.9487 - precision: 0.9185 - recall: 0.7688 - tn: 1838.3223 - tp: 3568.1848
Epoch 4/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8111 - auc: 0.9032 - fn: 991.6872 - fp: 304.8341 - loss: 0.3944 - prc: 0.9562 - precision: 0.9250 - recall: 0.7904 - tn: 1812.7203 - tp: 3673.9526
Epoch 5/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8097 - auc: 0.9066 - fn: 974.2465 - fp: 290.4502 - loss: 0.3863 - prc: 0.9